## Visualizado de los datos .json

In [1]:
import pandas as pd
ruta = (r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2019\20190520.json")
datos = pd.read_json(ruta, orient = 'records')
datos

,version,secret,type,data
apFloors,2.1,3Sm4rt1T2018,DevicesSeen,[]
apMac,2.1,3Sm4rt1T2018,DevicesSeen,xx:xx:xx:xx:xx:xx
apTags,2.1,3Sm4rt1T2018,DevicesSeen,"[groupname:IB01, hotspot:ibacapital, sensornam..."
observations,2.1,3Sm4rt1T2018,DevicesSeen,"[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


In [2]:
print(type(datos))

<class 'pandas.core.frame.DataFrame'>


Primera fila -> appFloors, muestra las columnas version, secret, type y data + Name con el nombre de la columna y el tipo

In [3]:
datos.iloc[0] #primera fila

version             2.1
secret     3Sm4rt1T2018
type        DevicesSeen
data                 []
Name: apFloors, dtype: object

Cuarta fila -> Observations, muestra las columnas version, secret, type y data + Name con el nombre de la columna y el tipo

In [4]:
data = datos.iloc[3] # fila observations
data

version                                                  2.1
secret                                          3Sm4rt1T2018
type                                             DevicesSeen
data       [{'ipv4': None, 'location': {'lat': 0.0, 'lng'...
Name: observations, dtype: object

A continuación aislamos la columna data de la fila de observaciones, dicha columna contiene todos los datos

In [5]:
columna_data = datos.iloc[:, [3]] #columna 3 (data)
columna_data

,data
apFloors,[]
apMac,xx:xx:xx:xx:xx:xx
apTags,"[groupname:IB01, hotspot:ibacapital, sensornam..."
observations,"[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


Array de datos (data) aislado

In [6]:
observations = columna_data.iloc[[3]]
observations

,data
observations,"[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


A continuación imprimimos el primer elemento de datos almacenado en el array de "data" en la fila de observaciones

In [7]:
#primer array de datos de observaciones
for client in observations['data']:
    cliente1 = client[0]
    
cliente1

{'ipv4': None,
 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []},
 'seenTime': '2019-05-20T11:50:59Z',
 'ssid': None,
 'os': None,
 'clientMac': '44:33:4c:b9:50:90',
 'seenEpoch': 1558353059,
 'rssi': 26,
 'ipv6': None,
 'manufacturer': ''}

In [49]:
print(type(cliente1))

<class 'dict'>


## Conectamos con la BBDD mongoDB en local

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
import json
import warnings
warnings.filterwarnings('ignore')

In [9]:
Client = MongoClient('localhost', 27017)
Client.list_database_names()

['TFG', 'admin', 'config', 'local']

In [10]:
db = Client.TFG
collect_names = db.list_collection_names()
collect_names

['DATA', 'Data']

In [11]:
client = MongoClient('localhost', 27017)
db = client['TFG']
collection_currency = db['Data']

In [12]:
print (collection_currency)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'TFG'), 'Data')


## Realizamos la ingesta de un único archivo a la BBDD

In [13]:
import json
from pymongo import MongoClient

nombre = "20190520.json"
archivo = "/"+ nombre
URL = ( r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2019" + archivo)

client = MongoClient('localhost', 27017)
db = client['TFG']
collection_currency = db['Data']

with open(URL) as f:
    file_data = json.load(f)

import traceback
try:
    result = collection_currency.insert_one(file_data) 
    print (result.inserted_id)
except Exception:
    traceback.print_exc()

client.close()

620a646dd4f34b216941e9b2


Visualizado de los datos en mongoDB

In [14]:
import pymongo
 
client = pymongo.MongoClient('localhost', 27017)
 
# Database Name
db = client["TFG"]
 
# Collection Name
col = db["Data"]
 
x = col.find_one()
print(x)

{'_id': ObjectId('620a642246b2f10567590f67'), 'version': '2.1', 'secret': '3Sm4rt1T2018', 'type': 'DevicesSeen', 'data': {'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [], 'apTags': ['groupname:IB01', 'hotspot:ibacapital', 'sensorname:ibacp18-0001', 'spot_id:ibacp18-001-001', 'country:ES', 'state:12', 'city:28', 'zipcode:28001', 'zone:2'], 'observations': [{'ipv4': None, 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []}, 'seenTime': '2019-05-20T11:50:59Z', 'ssid': None, 'os': None, 'clientMac': '44:33:4c:b9:50:90', 'seenEpoch': 1558353059, 'rssi': 26, 'ipv6': None, 'manufacturer': ''}, {'ipv4': None, 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []}, 'seenTime': '2019-05-20T11:50:59Z', 'ssid': None, 'os': None, 'clientMac': '88:ad:d2:8d:bf:78', 'seenEpoch': 1558353059, 'rssi': 25, 'ipv6': None, 'manufacturer': ''}, {'ipv4': None, 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []}, 'seenTime': '2019-05-20T11:50:59Z', 'ssid': None, 'os': None

## Ingesta masiva de datos del 2019 en mongoDB

A continuación listamos todos los archivos json de la ruta especificada

In [15]:
from pathlib import Path

path = (r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2019")
def ls3(path):
    return [obj.name for obj in Path(path).iterdir() if obj.is_file()]

archivos = ls3(path)
print (archivos)

['20190520.json', '20190521.json', '20190522.json', '20190523.json', '20190524.json', '20190525.json', '20190527.json', '20190528.json', '20190529.json', '20190530.json', '20190531.json', '20190603.json', '20190604.json', '20190605.json', '20190606.json', '20190607.json', '20190608.json', '20190610.json', '20190611.json', '20190612.json', '20190613.json', '20190614.json', '20190615.json', '20190616.json', '20190617.json', '20190618.json', '20190619.json', '20190620.json', '20190621.json', '20190622.json', '20190623.json', '20190624.json', '20190625.json', '20190626.json', '20190627.json', '20190628.json', '20190629.json', '20190630.json', '20190701.json', '20190702.json', '20190703.json', '20190704.json', '20190705.json', '20190706.json', '20190707.json', '20190708.json', '20190709.json', '20190710.json', '20190711.json', '20190712.json', '20190713.json', '20190714.json', '20190715.json', '20190716.json', '20190717.json', '20190718.json', '20190719.json', '20190720.json', '20190721.jso

In [16]:
archivos[1]

'20190521.json'

In [17]:
len(archivos)

216

Ingestamos los archivos json en la BBDD

In [18]:
import traceback

client = MongoClient('localhost', 27017)
db = client['TFG']
collection_currency = db['Data2019']

for n in archivos:
    nombre = n
    archivo = "/"+ nombre
    URL = ( r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2019" + archivo)
    with open(URL) as f:
        file_data = json.load(f)
    try:
        result = collection_currency.insert_one(file_data) 
        print (result.inserted_id)
    except Exception:
        traceback.print_exc()
        client.close()


620a646dd4f34b216941e9b5
620a646dd4f34b216941e9b6
620a646dd4f34b216941e9b7
620a646dd4f34b216941e9b8
620a646ed4f34b216941e9b9
620a646ed4f34b216941e9ba
620a646ed4f34b216941e9bb
620a646ed4f34b216941e9bc
620a646fd4f34b216941e9bd
620a646fd4f34b216941e9be
620a646fd4f34b216941e9bf
620a6470d4f34b216941e9c0
620a6470d4f34b216941e9c1
620a6470d4f34b216941e9c2
620a6471d4f34b216941e9c3
620a6471d4f34b216941e9c4
620a6471d4f34b216941e9c5
620a6472d4f34b216941e9c6
620a6472d4f34b216941e9c7
620a6472d4f34b216941e9c8
620a6472d4f34b216941e9c9
620a6473d4f34b216941e9ca
620a6473d4f34b216941e9cb
620a6474d4f34b216941e9cc
620a6474d4f34b216941e9cd
620a6475d4f34b216941e9ce
620a6475d4f34b216941e9cf
620a6476d4f34b216941e9d0
620a6477d4f34b216941e9d1
620a6477d4f34b216941e9d2
620a6478d4f34b216941e9d3
620a6478d4f34b216941e9d4
620a6479d4f34b216941e9d5
620a6479d4f34b216941e9d6
620a647ad4f34b216941e9d7
620a647bd4f34b216941e9d8
620a647bd4f34b216941e9d9
620a647cd4f34b216941e9da
620a647dd4f34b216941e9db
620a647dd4f34b216941e9dc


## Ingesta masiva de datos 2020 en mongoDB

In [19]:
from pathlib import Path

path = (r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2020")
def ls3(path):
    return [obj.name for obj in Path(path).iterdir() if obj.is_file()]

archivos2 = ls3(path)
print (archivos2)

['20200101.json', '20200102.json', '20200103.json', '20200107.json', '20200108.json', '20200109.json', '20200110.json', '20200113.json', '20200114.json', '20200115.json', '20200116.json', '20200117.json', '20200118.json', '20200119.json', '20200120.json', '20200121.json', '20200122.json', '20200123.json', '20200124.json', '20200125.json', '20200126.json', '20200127.json', '20200128.json', '20200129.json', '20200130.json', '20200131.json', '20200201.json', '20200202.json', '20200203.json', '20200204.json', '20200205.json', '20200206.json', '20200207.json', '20200208.json', '20200209.json', '20200210.json', '20200211.json', '20200212.json', '20200213.json', '20200214.json', '20200215.json', '20200216.json', '20200217.json', '20200218.json', '20200219.json', '20200220.json', '20200221.json', '20200222.json', '20200223.json', '20200224.json', '20200225.json', '20200226.json', '20200227.json', '20200228.json', '20200229.json', '20200301.json', '20200302.json', '20200303.json', '20200304.jso

In [20]:
archivos2[1]

'20200102.json'

In [21]:
len(archivos2)

237

In [22]:
import traceback
from pymongo import MongoClient
import json


client = MongoClient('localhost', 27017)
db = client['TFG']
collection_currency = db['Data2020']

for n in archivos2:
    nombre = n
    archivo = "/"+ nombre
    URL = ( r"D:\Álvaro Velázquez\Desktop\UNI\TFG\Datos\2020" + archivo)
    with open(URL) as f:
        file_data = json.load(f)
    try:
        result = collection_currency.insert_one(file_data) 
        print (result.inserted_id)
    except Exception:
        traceback.print_exc()
        client.close()

620a64fbd4f34b216941ea8e
620a64fbd4f34b216941ea8f
620a64fbd4f34b216941ea90
620a64fbd4f34b216941ea91
620a64fcd4f34b216941ea92
620a64fcd4f34b216941ea93
620a64fcd4f34b216941ea94
620a64fcd4f34b216941ea95
620a64fcd4f34b216941ea96
620a64fdd4f34b216941ea97
620a64fed4f34b216941ea98
620a6500d4f34b216941ea99


Traceback (most recent call last):
  File "C:\Users\alvar\AppData\Local\Temp/ipykernel_2472/260832613.py", line 17, in <module>
    result = collection_currency.insert_one(file_data)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 695, in insert_one
    self._insert(document,
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 610, in _insert
    return self._insert_one(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 599, in _insert_one
    self.__database.client._retryable_write(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1491, in _retryable_write
    return self._retry_with_session(retryable, func, s, None)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1384, in _retry_wit

620a6508d4f34b216941ea9c
620a650ad4f34b216941ea9d


Traceback (most recent call last):
  File "C:\Users\alvar\AppData\Local\Temp/ipykernel_2472/260832613.py", line 17, in <module>
    result = collection_currency.insert_one(file_data)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 695, in insert_one
    self._insert(document,
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 610, in _insert
    return self._insert_one(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 599, in _insert_one
    self.__database.client._retryable_write(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1491, in _retryable_write
    return self._retry_with_session(retryable, func, s, None)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1384, in _retry_wit

620a6558d4f34b216941eab7
620a655ad4f34b216941eab8
620a655bd4f34b216941eab9
620a655dd4f34b216941eaba
620a655ed4f34b216941eabb
620a655fd4f34b216941eabc
620a6561d4f34b216941eabd
620a6563d4f34b216941eabe
620a6564d4f34b216941eabf
620a6566d4f34b216941eac0
620a6567d4f34b216941eac1
620a6568d4f34b216941eac2
620a656ad4f34b216941eac3
620a656bd4f34b216941eac4


Traceback (most recent call last):
  File "C:\Users\alvar\AppData\Local\Temp/ipykernel_2472/260832613.py", line 17, in <module>
    result = collection_currency.insert_one(file_data)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 695, in insert_one
    self._insert(document,
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 610, in _insert
    return self._insert_one(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\collection.py", line 599, in _insert_one
    self.__database.client._retryable_write(
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1491, in _retryable_write
    return self._retry_with_session(retryable, func, s, None)
  File "C:\Users\alvar\AppData\Local\Programs\Python\Python310\lib\site-packages\pymongo\mongo_client.py", line 1384, in _retry_wit

620a657ad4f34b216941eaca
620a657cd4f34b216941eacb
620a657dd4f34b216941eacc
620a657fd4f34b216941eacd
620a6581d4f34b216941eace
620a6583d4f34b216941eacf
620a6585d4f34b216941ead0
620a6586d4f34b216941ead1
620a6586d4f34b216941ead2
620a6587d4f34b216941ead3
620a6587d4f34b216941ead4
620a6587d4f34b216941ead5
620a6588d4f34b216941ead6
620a6588d4f34b216941ead7
620a6588d4f34b216941ead8
620a6589d4f34b216941ead9
620a6589d4f34b216941eada
620a658ad4f34b216941eadb
620a658ad4f34b216941eadc
620a658bd4f34b216941eadd
620a658bd4f34b216941eade
620a658bd4f34b216941eadf
620a658cd4f34b216941eae0
620a658cd4f34b216941eae1
620a658cd4f34b216941eae2
620a658dd4f34b216941eae3
620a658dd4f34b216941eae4
620a658ed4f34b216941eae5
620a658ed4f34b216941eae6
620a658fd4f34b216941eae7
620a6590d4f34b216941eae8
620a6591d4f34b216941eae9
620a6592d4f34b216941eaea
620a6593d4f34b216941eaeb
620a6595d4f34b216941eaec
620a6596d4f34b216941eaed
620a6597d4f34b216941eaee
620a6597d4f34b216941eaef
620a6598d4f34b216941eaf0
620a6599d4f34b216941eaf1


## Limpieza de datos 

Primero hay que eliminar los datos que no se van a usar.

Para ello voy a quedarme con los datos útiles y estos los voy a guardar por separado para su posterior análisis

Cómo he planteado la prueba de limpieza de datos:

    Primero-> Hacemos una llamada a la BBDD para traernos un único bloque de datos.
    
    Segundo-> Ese primer bloque de datos lo convertimos a formato DataFrame de Pandas
    
    Tercero-> Una vez lo tenemos en formato DataFrame, eliminamos las filas que no nos sirven para el análisis de datos 
        (_id, version, secret, type)
        
    Cuarto-> Aislamos la columna que queremos analizar (observations) la cual contiene toda la información.
    
    Quinto-> Convertimos todos los elementos de la columna en filas de elementos independientes
    
    sexto-> Cada fila se convierte en un diccionario de python y podemos sacar el valor de los datos.

In [1]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["TFG"]
mycol = mydb["Data2019"]

x = mycol.find_one()

Primeros_datos = []
Primeros_datos = x
Primeros_datos

{'_id': ObjectId('620a646dd4f34b216941e9b5'),
 'version': '2.1',
 'secret': '3Sm4rt1T2018',
 'type': 'DevicesSeen',
 'data': {'apMac': 'xx:xx:xx:xx:xx:xx',
  'apFloors': [],
  'apTags': ['groupname:IB01',
   'hotspot:ibacapital',
   'sensorname:ibacp18-0001',
   'spot_id:ibacp18-001-001',
   'country:ES',
   'state:12',
   'city:28',
   'zipcode:28001',
   'zone:2'],
  'observations': [{'ipv4': None,
    'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []},
    'seenTime': '2019-05-20T11:50:59Z',
    'ssid': None,
    'os': None,
    'clientMac': '44:33:4c:b9:50:90',
    'seenEpoch': 1558353059,
    'rssi': 26,
    'ipv6': None,
    'manufacturer': ''},
   {'ipv4': None,
    'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []},
    'seenTime': '2019-05-20T11:50:59Z',
    'ssid': None,
    'os': None,
    'clientMac': '88:ad:d2:8d:bf:78',
    'seenEpoch': 1558353059,
    'rssi': 25,
    'ipv6': None,
    'manufacturer': ''},
   {'ipv4': None,
    'location': 

In [2]:
print(type(Primeros_datos))

<class 'dict'>


In [3]:
for n in Primeros_datos:
    print (n)


_id
version
secret
type
data


El siguiente script me devuelve todos los datos de mongoDB de 2019.

Nota: no ejecutar, se peta

Para poder aumentar la salida de datos, debemos abrir un nuevo cmd y escribir:

jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

---

import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")


mydb = myclient["TFG"]


mycol = mydb["Data2019"]


for x in mycol.find():


    print(x)
    

Creamos un data frame de pandas para gestionar los datos de la BBDD, en este ejemplo estamos trabajando con el primer bloque de datos de la BBDD de 2019

In [4]:
import pandas as pd

In [5]:
print(pd.DataFrame(list(Primeros_datos.items()),
                   columns=['Name', 'Data']))

      Name                                               Data
0      _id                           620a646dd4f34b216941e9b5
1  version                                                2.1
2   secret                                       3Sm4rt1T2018
3     type                                        DevicesSeen
4     data  {'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],...


In [6]:
data = pd.DataFrame(list(Primeros_datos.items()),columns=['Name', 'Data'])
data

,Name,Data
0,_id,620a646dd4f34b216941e9b5
1,version,2.1
2,secret,3Sm4rt1T2018
3,type,DevicesSeen
4,data,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [7]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


A continuación con el método Drop, eliminamos las filas que no nos interesan para el análisis

In [8]:
Only_data = data.drop([0,1,2,3], axis = 0)
Only_data

,Name,Data
4,data,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [9]:
DATA = Only_data.iloc[:, [1]] #columna Data
DATA

,Data
4,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [10]:
print(type(DATA))

<class 'pandas.core.frame.DataFrame'>


In [11]:
DATA

,Data
4,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [12]:
print(type(DATA))

<class 'pandas.core.frame.DataFrame'>


Al tener un data frame de pandas podemos aplicar el método explode para separar los campos

In [13]:
NewDatas = DATA.explode("Data")
NewDatas

,Data
4,apMac
4,apFloors
4,apTags
4,observations


En la siguiente líena convertimos el pandas Data Frame en Json file

In [27]:
#DATA.to_json(r'dattaas.json')

https://ichi.pro/es/extraccion-de-datos-analice-un-objeto-json-anidado-de-3-y-conviertalo-en-un-marco-de-datos-pandas-39357327828653

In [15]:
import pandas as pd
df_final = (
    pd.DataFrame(DATA['Data'].apply(pd.Series))
)
df_final

,apMac,apFloors,apTags,observations
4,xx:xx:xx:xx:xx:xx,[],"[groupname:IB01, hotspot:ibacapital, sensornam...","[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


A continuación eliminamos las columnas que no necesitamos (apMac, apFloors, apTags)

In [16]:
observations = df_final.drop(['apMac', 'apFloors', 'apTags'], axis=1)
observations

,observations
4,"[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


 continuación nos quedamos con los valores que realmente necesitamos separados en filas

In [17]:
NewDatas2 = observations.explode("observations")
NewDatas2

,observations
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
...,...
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."


In [18]:
print(type(NewDatas2))

<class 'pandas.core.frame.DataFrame'>


In [19]:
len(NewDatas2)

4064

Cada fila de mi data frame es un dic (diccionario de python)

In [20]:
datos_1 = NewDatas2.iloc[0]['observations']
datos_1

{'ipv4': None,
 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []},
 'seenTime': '2019-05-20T11:50:59Z',
 'ssid': None,
 'os': None,
 'clientMac': '44:33:4c:b9:50:90',
 'seenEpoch': 1558353059,
 'rssi': 26,
 'ipv6': None,
 'manufacturer': ''}

In [21]:
print (type(datos_1))

<class 'dict'>


Para acceder a un elemento de un diccionario se hace de la siguiente manera:

In [22]:
clientMac = datos_1['clientMac']
clientMac

'44:33:4c:b9:50:90'

## recopilación de todos los datos útiles de 2019

Conseguir crear un array de datos por tipo de datos:
        
        Array con todas las direcciones MAC
        
        Array con la señal rssi
        
        Array con seeTime
        
        Array con seeRpoch
        

creación de los arrays:

In [23]:
Direcciones_Mac = []
señal_rssi = []
seenTime = []
seenEpoch = []

En el siguiente bloque de código extraemos los datos que queremos para analizar

In [24]:
ids = -1
index = len(NewDatas2)

for i in range(index):
    ids = ids + 1
    datos= NewDatas2.iloc[ids]['observations']
    clientMac = datos['clientMac']
    rssi = datos['rssi']
    seentime = datos['seenTime']
    seenepoch = datos['seenEpoch']
    
    Direcciones_Mac.append(clientMac)
    señal_rssi.append(rssi)
    seenTime.append(seentime)
    seenEpoch.append(seenepoch)

In [26]:
Direcciones_Mac

['44:33:4c:b9:50:90',
 '88:ad:d2:8d:bf:78',
 'd8:61:94:f5:ea:22',
 '44:33:4c:b9:56:38',
 '44:33:4c:b9:3a:b9',
 '88:ad:d2:8d:bf:78',
 'd8:61:94:f5:ea:22',
 '34:fe:3d:eb:45:9b',
 '44:33:4c:b9:56:38',
 '44:33:4c:b9:50:90',
 '44:33:4c:b9:3a:b9',
 '5c:41:5a:bb:f1:d8',
 'cc:9e:a2:cc:80:ef',
 'd8:6c:63:77:5d:b8',
 '3c:5c:c4:8d:3b:f8',
 'cc:9e:a2:7c:a6:5a',
 'cc:f7:35:72:ed:c1',
 '34:fe:3d:eb:45:9b',
 'd8:61:94:f5:ea:22',
 '44:33:4c:b9:3a:b9',
 '44:33:4c:b9:50:90',
 '44:33:4c:b9:56:38',
 '14:99:e2:50:74:55',
 'd8:61:94:f5:ea:22',
 '34:fe:3d:eb:45:9b',
 'cc:f7:35:72:ed:c1',
 '44:33:4c:b9:3a:b9',
 '5c:41:5a:bb:f1:d8',
 '3c:5c:c4:8d:3b:f8',
 'e0:ac:cb:5c:ad:cc',
 'd8:6c:63:77:5d:b8',
 '44:33:4c:b9:3a:b9',
 'cc:9e:a2:7c:a6:5a',
 '34:fe:3d:eb:45:9b',
 'd8:61:94:f5:ea:22',
 '44:33:4c:b9:56:38',
 'cc:f7:35:72:ed:c1',
 '44:33:4c:b9:50:90',
 'da:a1:19:37:d6:8a',
 'f0:98:9d:bd:48:65',
 '5c:41:5a:bb:f1:d8',
 'da:a1:19:a0:08:f3',
 '14:99:e2:50:74:55',
 '44:33:4c:b9:56:38',
 'd8:61:94:f5:ea:22',
 '44:33:4c

In [27]:
len(Direcciones_Mac)

4064

In [149]:
señal_rssi

[25,
 25,
 29,
 24,
 40,
 23,
 24,
 27,
 26,
 31,
 35,
 26,
 29,
 29,
 33,
 36,
 25,
 25,
 34,
 29,
 29,
 27,
 59,
 30,
 31,
 30,
 40,
 27,
 35,
 63,
 31,
 35,
 39,
 28,
 33,
 35,
 27,
 31,
 35,
 41,
 28,
 26,
 61,
 32,
 24,
 31,
 36,
 27,
 33,
 29,
 25,
 28,
 23,
 27,
 23,
 24,
 33,
 33,
 34,
 25,
 37,
 40,
 35,
 30,
 60,
 28,
 28,
 38,
 31,
 28,
 29,
 24,
 24,
 27,
 38,
 34,
 26,
 39,
 40,
 23,
 36,
 32,
 27,
 37,
 39,
 27,
 26,
 40,
 24,
 33,
 30,
 33,
 44,
 37,
 32,
 28,
 36,
 39,
 28,
 28,
 35,
 32,
 36,
 33,
 29,
 26,
 34,
 28,
 23,
 24,
 31,
 22,
 35,
 32,
 24,
 33,
 34,
 24,
 26,
 33,
 38,
 61,
 43,
 30,
 66,
 29,
 32,
 24,
 37,
 27,
 29,
 33,
 29,
 42,
 30,
 33,
 28,
 34,
 23,
 23,
 31,
 25,
 39,
 26,
 31,
 35,
 38,
 25,
 23,
 30,
 31,
 33,
 40,
 32,
 30,
 22,
 26,
 28,
 31,
 27,
 23,
 22,
 44,
 41,
 28,
 32,
 28,
 36,
 25,
 35,
 33,
 28,
 29,
 40,
 27,
 38,
 27,
 39,
 27,
 24,
 25,
 27,
 28,
 35,
 26,
 34,
 22,
 25,
 39,
 34,
 28,
 29,
 29,
 37,
 37,
 30,
 30,
 42,
 42,
 24,


In [150]:
seenTime

['2019-05-20T11:50:59Z',
 '2019-05-20T11:50:59Z',
 '2019-05-20T11:50:59Z',
 '2019-05-20T11:51:00Z',
 '2019-05-20T11:51:05Z',
 '2019-05-20T11:52:03Z',
 '2019-05-20T11:52:05Z',
 '2019-05-20T11:52:07Z',
 '2019-05-20T11:52:09Z',
 '2019-05-20T11:52:10Z',
 '2019-05-20T11:52:10Z',
 '2019-05-20T11:52:11Z',
 '2019-05-20T11:52:14Z',
 '2019-05-20T11:52:19Z',
 '2019-05-20T11:52:22Z',
 '2019-05-20T11:52:23Z',
 '2019-05-20T11:52:42Z',
 '2019-05-20T11:56:25Z',
 '2019-05-20T11:56:26Z',
 '2019-05-20T11:56:27Z',
 '2019-05-20T11:57:28Z',
 '2019-05-20T11:57:28Z',
 '2019-05-20T11:57:28Z',
 '2019-05-20T11:57:28Z',
 '2019-05-20T11:57:28Z',
 '2019-05-20T11:57:31Z',
 '2019-05-20T11:57:31Z',
 '2019-05-20T11:57:32Z',
 '2019-05-20T11:57:33Z',
 '2019-05-20T11:57:37Z',
 '2019-05-20T11:58:23Z',
 '2019-05-20T11:58:24Z',
 '2019-05-20T11:58:24Z',
 '2019-05-20T11:58:24Z',
 '2019-05-20T11:58:27Z',
 '2019-05-20T11:58:30Z',
 '2019-05-20T11:58:30Z',
 '2019-05-20T11:58:40Z',
 '2019-05-20T11:58:42Z',
 '2019-05-20T11:58:49Z',


In [151]:
seenEpoch

[1558353059,
 1558353059,
 1558353059,
 1558353060,
 1558353065,
 1558353123,
 1558353125,
 1558353127,
 1558353129,
 1558353130,
 1558353130,
 1558353131,
 1558353134,
 1558353139,
 1558353142,
 1558353143,
 1558353162,
 1558353385,
 1558353386,
 1558353387,
 1558353448,
 1558353448,
 1558353448,
 1558353448,
 1558353448,
 1558353451,
 1558353451,
 1558353452,
 1558353453,
 1558353457,
 1558353503,
 1558353504,
 1558353504,
 1558353504,
 1558353507,
 1558353510,
 1558353510,
 1558353520,
 1558353522,
 1558353529,
 1558353532,
 1558353544,
 1558353549,
 1558353781,
 1558353781,
 1558353782,
 1558353840,
 1558353843,
 1558353843,
 1558353843,
 1558353843,
 1558353846,
 1558353847,
 1558353847,
 1558353847,
 1558353851,
 1558353853,
 1558353860,
 1558353869,
 1558353869,
 1558353887,
 1558353898,
 1558353905,
 1558353908,
 1558353911,
 1558353911,
 1558353913,
 1558353915,
 1558353917,
 1558353939,
 1558354301,
 1558354301,
 1558354302,
 1558354308,
 1558354308,
 1558354308,
 1558354308,

A continuación exportamos los datos en csv, para tener los datos que vamos a analizar en un único documento

In [156]:
import pandas as pd

data = {'Direcciones_Mac': Direcciones_Mac,
        'rssi_signal': señal_rssi,
        'seenTime': seenTime,
        'seenEpoch': seenEpoch}

df = pd.DataFrame(data, columns = ['Direcciones_Mac', 'rssi_signal', 'seenTime', 'seenEpoch'])
#df.to_csv(r"C:\Users\alvar\OneDrive\Escritorio\Datos_Para_Análisis.csv")
df.to_csv(r"Datos_Para_Análisis.csv")

In [160]:
import pandas as pd
 
datos = pd.read_csv(r"C:\Users\alvar\OneDrive\Escritorio\Datos_Para_Análisis.csv")
datos

,index;Direcciones_Mac;rssi_signal;seenTime;seenEpoch
0,0;88:ad:d2:8d:bf:78;25;2019-05-20T11:50:59Z;15...
1,1;88:ad:d2:8d:bf:78;25;2019-05-20T11:50:59Z;15...
2,2;d8:61:94:f5:ea:22;29;2019-05-20T11:50:59Z;15...
3,3;44:33:4c:b9:56:38;24;2019-05-20T11:51:00Z;15...
4,4;44:33:4c:b9:3a:b9;40;2019-05-20T11:51:05Z;15...
...,...
12187,12187;34:fe:3d:eb:45:9b;32;2019-05-20T21:48:48...
12188,12188;44:33:4c:b9:3a:b9;32;2019-05-20T21:48:49...
12189,12189;5c:41:5a:bb:f1:d8;23;2019-05-20T21:48:51...
12190,12190;50:8f:4c:c1:d0:9f;31;2019-05-20T21:48:55...


### Nota: hasta ahora los datos que hemos recopilado son los correspondientes al primer objeto de la BBDD de 
### Data2019

## Extracción de todos los objetos de la colección de 2019

En el siguiente script nos vamos a traer todos los datos de 2019

In [29]:
from pymongo import MongoClient
import pandas as pd
 
cliente = MongoClient ('localhost', 27017)
collection = cliente ["TFG"] ["Data2019"]
data = collection.find()
data = list (data) # Al convertir a una lista, puede filtrar solo los datos que necesita según la situación. (para filtrado transversal)
df = pd.DataFrame (data) # leer la tabla completa (DataFrame)

In [30]:
df

,_id,version,secret,type,data
0,620a646dd4f34b216941e9b5,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
1,620a646dd4f34b216941e9b6,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
2,620a646dd4f34b216941e9b7,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
3,620a646dd4f34b216941e9b8,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
4,620a646ed4f34b216941e9b9,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
...,...,...,...,...,...
211,620a64fad4f34b216941ea88,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
212,620a64fad4f34b216941ea89,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
213,620a64fad4f34b216941ea8a,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."
214,620a64fad4f34b216941ea8b,2.1,3Sm4rt1T2018,DevicesSeen,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [31]:
datos = df.iloc[215]
datos

_id                                 620a64fad4f34b216941ea8c
version                                                  2.1
secret                                          3Sm4rt1T2018
type                                             DevicesSeen
data       {'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],...
Name: 215, dtype: object

In [32]:
data = pd.DataFrame(list(datos.items()),columns=['Name', 'Data'])
data

,Name,Data
0,_id,620a64fad4f34b216941ea8c
1,version,2.1
2,secret,3Sm4rt1T2018
3,type,DevicesSeen
4,data,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [33]:
Only_data = data.drop([0,1,2,3], axis = 0)
Only_data

,Name,Data
4,data,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [34]:
DATA = Only_data.iloc[:, [1]] #columna Data
DATA

,Data
4,"{'apMac': 'xx:xx:xx:xx:xx:xx', 'apFloors': [],..."


In [35]:
NewDatas = DATA.explode("Data")
NewDatas

,Data
4,apMac
4,apFloors
4,apTags
4,observations


In [36]:
import pandas as pd
df_final = (pd.DataFrame(DATA['Data'].apply(pd.Series)))
df_final

,apMac,apFloors,apTags,observations
4,xx:xx:xx:xx:xx:xx,[],"[groupname:IB01, hotspot:ibacapital, sensornam...","[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


In [37]:
observations = df_final.drop(['apMac', 'apFloors', 'apTags'], axis=1)
observations

,observations
4,"[{'ipv4': None, 'location': {'lat': 0.0, 'lng'..."


In [38]:
NewDatas2 = observations.explode("observations")
NewDatas2

,observations
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
...,...
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."
4,"{'ipv4': None, 'location': {'lat': 0.0, 'lng':..."


In [39]:
datos_1 = NewDatas2.iloc[1000]['observations']
datos_1

{'ipv4': None,
 'location': {'lat': 0.0, 'lng': 0.0, 'unc': 0.0, 'x': [], 'y': []},
 'seenTime': '2019-12-31T08:02:56Z',
 'ssid': None,
 'os': None,
 'clientMac': '70:c9:4e:c3:c5:cf',
 'seenEpoch': 1577779376,
 'rssi': 22,
 'ipv6': None,
 'manufacturer': ''}

In [40]:
clientMac = datos_1['clientMac']
clientMac

'70:c9:4e:c3:c5:cf'

El siguiente script imprime sólo el primer elemento de cada objeto de la base de datos, hay que conseguir combinar este escript que recorre todos los objetos con el escript que recorre todas las observaciones.

In [41]:
import pandas as pd

Direcciones_Mac = []
señal_rssi = []
seenTime = []
seenEpoch = []

index = 215
ids = -1

for i in range(index):
    ids = ids + 1
    datos = df.iloc[ids]
    data = pd.DataFrame(list(datos.items()),columns=['Name', 'Data'])
    Only_data = data.drop([0,1,2,3], axis = 0)
    DATA = Only_data.iloc[:, [1]] #columna Data
    NewDatas = DATA.explode("Data")
    df_final = (pd.DataFrame(DATA['Data'].apply(pd.Series)))
    observations = df_final.drop(['apMac', 'apFloors', 'apTags'], axis=1)
    NewDatas2 = observations.explode("observations")
    datos_x = NewDatas2.iloc[ids]['observations']
    clientMac = datos_x['clientMac']
    Direcciones_Mac.append(clientMac)
    rssi = datos_x['rssi']
    señal_rssi.append(rssi)
    seentime = datos_x['seenTime']
    seenTime.append(seentime)
    seenepoch = datos_x['seenEpoch']
    seenEpoch.append(seenepoch)
    

data = {'Direcciones_Mac': Direcciones_Mac,
        'rssi_signal': señal_rssi,
        'seenTime': seenTime,
        'seenEpoch': seenEpoch}

df = pd.DataFrame(data, columns = ['Direcciones_Mac', 'rssi_signal', 'seenTime', 'seenEpoch'])
#df.to_csv(r"C:\Users\alvar\OneDrive\Escritorio\Datos_Para_Análisis.csv")
df.to_csv(r"Datos_Para_Análisis2.csv")

datos = pd.read_csv("Datos_Para_Análisis2.csv")
datos

,Unnamed: 0,Direcciones_Mac,rssi_signal,seenTime,seenEpoch
0,0,44:33:4c:b9:50:90,26,2019-05-20T11:50:59Z,1558353059
1,1,44:33:4c:b9:56:38,38,2019-05-20T22:01:12Z,1558389672
2,2,a6:ee:30:78:9a:c4,23,2019-05-21T22:01:30Z,1558476090
3,3,cc:21:19:25:05:57,27,2019-05-22T22:00:23Z,1558562423
4,4,74:eb:80:ab:3e:b2,23,2019-05-23T22:00:11Z,1558648811
...,...,...,...,...,...
210,210,70:c9:4e:c3:c5:cf,22,2019-12-26T01:23:01Z,1577323381
211,211,48:e2:44:d3:ce:e7,28,2019-12-26T23:10:34Z,1577401834
212,212,9a:3b:d6:e2:d4:a8,24,2019-12-27T22:38:46Z,1577486326
213,213,da:a1:19:ba:12:c2,22,2019-12-28T22:39:29Z,1577572769


In [46]:
ids = -1
index = len(NewDatas2)

for i in range(index):
    ids = ids + 1
    datos= NewDatas2.iloc[ids]['observations']
    clientMac = datos['clientMac']
    rssi = datos['rssi']
    seentime = datos['seenTime']
    seenepoch = datos['seenEpoch']
    
    Direcciones_Mac.append(clientMac)
    señal_rssi.append(rssi)
    seenTime.append(seentime)
    seenEpoch.append(seenepoch)

In [47]:
Direcciones_Mac

['16:69:07:6c:71:a8',
 'e2:49:b7:ab:71:12',
 'a0:28:ed:78:7c:6e',
 'e4:0e:ee:fd:8e:bb',
 'da:a1:19:d0:4b:ff',
 'da:a1:19:a5:57:0b',
 '1c:15:1f:ca:ff:66',
 '6a:6c:62:7e:e3:6b',
 '1c:15:1f:ca:ff:66',
 'ea:0a:c6:e3:1c:a3',
 'e4:0e:ee:fd:8e:bb',
 'da:a1:19:c3:3c:43',
 '02:6e:bf:12:47:50',
 '5a:0f:9f:fd:6c:09',
 'b4:cd:27:a2:78:ff',
 '0a:ae:6f:65:74:1e',
 '38:30:f9:46:8b:c1',
 'ce:7e:83:b5:3c:a7',
 'f6:30:ba:15:31:d6',
 'be:bf:c9:05:71:aa',
 '1e:4a:39:91:a5:87',
 '6a:5a:11:c4:0d:8a',
 'ba:d6:2b:8d:68:b1',
 'a2:ab:1f:dc:45:8f',
 '0e:96:94:32:a9:5f',
 '52:b5:e1:67:cd:ae',
 'aa:a5:11:f7:e5:b5',
 'd2:76:fc:3f:a0:e0',
 'da:a1:19:6a:1d:ea',
 '32:ac:fe:d9:04:c8',
 'da:a1:19:33:1c:c1',
 '9e:ab:0b:df:b2:6c',
 'da:a5:a3:58:1a:27',
 'da:a1:19:4f:d4:c0',
 '54:b1:21:25:47:a3',
 '54:b1:21:25:47:a3',
 '3c:dc:bc:0a:21:2a',
 '3c:dc:bc:0a:21:2a',
 'bc:a5:8b:c2:30:e2',
 '24:2e:02:d1:8e:b3',
 'c6:03:a6:4a:d6:b8',
 '36:49:78:18:b2:81',
 '4a:1b:67:c6:77:69',
 '3a:9f:2c:73:a2:0d',
 '24:2e:02:d1:8e:b3',
 'da:a1:19

In [48]:
len(Direcciones_Mac)

6488